# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json
from pprint import pprint

from sklearn.preprocessing import LabelEncoder

In [2]:
# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [151]:
display(portfolio.shape)
portfolio # in total 10 unique offers

(10, 6)

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [4]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [5]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [6]:
print( "Are there any null values in portfolio?", True in list(portfolio.isnull().any())) # ...no null values

Are there any null values in portfolio? False


In [15]:
def prepare_portfolio(portfolio):
    
    portfolio = portfolio.copy() # do not alter the original values outside of this function
    
    # get all possible channel_types
    channel_types = set()
    for l in portfolio.channels.to_list():
        channel_types.update(l)

    # create marker columns for each channel type. 1 exists, 0 does not exists. 0-1 availability matrix.
    for channel_type in channel_types:
        portfolio['ch_' + channel_type] = portfolio['channels'].apply(lambda x: 1 if channel_type in x else 0)
    
    # convert offer_type from string to 0-1 availability matrix
    offer_type_dummies = pd.get_dummies(portfolio.offer_type, prefix='ot_')
    portfolio = pd.concat([portfolio, offer_type_dummies], axis=1, sort=False)
    
    # encode offer_type from string to int
    le = LabelEncoder()
    portfolio['offer_type_encoded'] = portfolio[['offer_type']].apply(le.fit_transform)
    
    # remove columns that have been converted
    portfolio.drop(['channels'], axis=1, inplace=True)
    
    # create an information dictionary of converted columns
    portfolio_info = {'channel_types': list(channel_types),
                      'offer_types:': set(dict(portfolio.offer_type.value_counts()).keys()),
                       'offer_type encoding': dict(zip(le.classes_, le.transform(le.classes_)))}
    return portfolio, portfolio_info

prep_portfolio, portfolio_info = prepare_portfolio(portfolio)
pprint(portfolio_info)
prep_portfolio.head()


{'channel_types': ['mobile', 'web', 'email', 'social'],
 'offer_type encoding': {'bogo': 0, 'discount': 1, 'informational': 2},
 'offer_types:': {'informational', 'bogo', 'discount'}}


,reward,difficulty,duration,offer_type,id,ch_mobile,ch_web,ch_email,ch_social,ot__bogo,ot__discount,ot__informational,offer_type_encoded
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,0,1,1,1,0,0,0
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,1,0,0,0
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,1,0,0,0,1,2
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,1,0,1,0,0,0
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,1,0,0,1,0,1


In [10]:
print( "Are there any null values in profile?", True in list(profile.isnull().any())) # ...no null values
display(profile.isnull().any())# ... in gender and income
profile.shape

Are there any null values in profile? True


gender               True
age                 False
id                  False
became_member_on    False
income               True
dtype: bool

(17000, 5)

In [11]:
print("#null_gender", profile[profile.gender.isna()].shape[0])
print("#null_income", profile[profile.income.isna()].shape[0])
# both row count with null values is 2175. Are they the same?

#null_gender 2175
#null_income 2175


In [12]:
print("Where gender is NaN then income is also Nan?", list(profile.gender.isna()) == list(profile.income.isna()))

Where gender is NaN then income is also Nan? True


In [13]:
print(profile.age.value_counts())
print("Age of 118 has exactly 2175 hits. The same count as number of NaNs. So are they the same rows?")
print("- " + str(profile[profile.age == 118].income.isnull().count() == 2175))

118    2175
58      408
53      372
51      363
54      359
       ... 
100      12
96        8
98        5
99        5
101       5
Name: age, Length: 85, dtype: int64
Age of 118 has exactly 2175 hits. The same count as number of NaNs. So are they the same rows?
- True


In [16]:
def prepare_profile(profile):
    profile = profile.copy()  # do not alter the original values outside of this function
    
    # correct highly possible fake value with age==118
    profile.age = profile.age.apply(lambda x: np.nan if x == 118 else x) # reset age==118 to NaN
    print("Number of rows with age 118 after setting age=NaN: " + str(profile[profile.age == 118].income.isnull().count()))
    
    # convert int date to real datetime
    profile['membership_startdate'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')
    
    return profile

prep_profile = prepare_profile(profile)
prep_profile.head()

Number of rows with age 118 after setting age=NaN: 0


,gender,age,id,became_member_on,income,membership_startdate
0,None,NaN,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,2017-02-12
1,F,55.0,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0,2017-07-15
2,None,NaN,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN,2018-07-12
3,F,75.0,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,2017-05-09
4,None,NaN,a03223e636434f42ac4c3df47e8bac43,20170804,NaN,2017-08-04


In [17]:
print( "Are there any null values in transcript?", True in list(transcript.isnull().any())) # ...no null values

Are there any null values in transcript? False


In [18]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [92]:
# the value column still has nested json encoded. Lets' analyse that structure...

def analyse_transcript(transcript):
    transcript = transcript.copy() # do not alter the original values outside of this function
    # find all possible value json keys and store it in a new column `value_type`
    transcript_value_keys = set()
    for d in transcript.value:
        transcript_value_keys.update(d.keys())
    print("value json keys: ", transcript_value_keys)
    transcript['value_type'] = transcript.value.apply(lambda d: next(iter(d.keys())))
    transcript['value_num_keys'] = transcript.value.apply(lambda d: len(d.keys()))    
    print("max number of value json keys: ", transcript.value.apply(lambda d: len(d.keys())).max())
    # there are json encoded values with more than 1 key. let's find all the possible key combinations...
    json_key_combinations = set()
    for _, row in transcript[transcript['value_num_keys'] > 1].iterrows():
        #print(row['value'])
        #print("hae", set(dict(row['value'])))
        json_key_combinations.update(set(dict(row['value'])))
        break

    print("all json key combinations:")
    pprint(json_key_combinations) # only 1 combination: {'offer_id', 'reward'}
                                                         
    print("all value json keys: ", transcript_value_keys)

    # find all event (string) types
    event_types = set(dict(transcript.event.value_counts()).keys())
    print("All event types: ", event_types)

    # event and value_types seem to be related.
    # lets create a mapping table/dict to seee which json keys corresponds to an event type
    event_value_mapping = {}
    for et in event_types:
        event_value_mapping[et] = set(transcript[transcript.event == et].value.apply(lambda x: ','.join(list(x.keys()))))
    pprint(event_value_mapping)
    # difference between `offer_id` and `offer id` is that `offer_id` corresponds to event type `offer_completed`
    # and therefore the json value includes a second attribute `reward`
    
    display(transcript.head())


analyse_transcript(transcript)

value json keys:  {'reward', 'offer_id', 'offer id', 'amount'}
max number of value json keys:  2
all json key combinations:
{'offer_id', 'reward'}
all value json keys:  {'reward', 'offer_id', 'offer id', 'amount'}
All event types:  {'transaction', 'offer viewed', 'offer completed', 'offer received'}
{'offer completed': {'offer_id,reward'},
 'offer received': {'offer id'},
 'offer viewed': {'offer id'},
 'transaction': {'amount'}}


,person,event,value,time,value_type,value_num_keys
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,offer id,1
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,offer id,1
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,offer id,1
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,offer id,1
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,offer id,1


In [131]:
def prepare_transcript(transcript):
    transcript = transcript.copy() # do not alter the original values outside of this function
    # create new columns each representing the json value for the corresponding event type.
    transcript['offer_id'] = transcript[
        transcript.event.isin(['offer received', 'offer reviewed', 'offer completed'])
        ]['value'].apply(lambda x: x['offer_id'] if ('offer_id' in x) else x['offer id'])
    transcript['reward'] = transcript[transcript.event == 'offer completed']['value'].apply(lambda x: x['reward'])
    transcript['amount'] = transcript[transcript.event == 'transaction']['value'].apply(lambda x: x['amount'])
    # remove columns that have been converted
    transcript.drop(['value'], axis=1, inplace=True)
    return transcript
    
prep_transcript = prepare_transcript(transcript)
prep_transcript.shape
#prep_transcript.groupby(by=['person','offer_id']).person.value_counts()

(306534, 6)

In [148]:
print("#offer_id not null", prep_transcript[prep_transcript['offer_id'].notna()].person.count())
print("#unique offer_id", prep_transcript.offer_id.nunique())
print("#portfolio offer_id", prep_portfolio.id.nunique())
# 10 offers were presented to 109856 customers

#offer_id not null 109856
#unique offer_id 10
#portfolio offer_id 10
